# **1. Import Library**

Pada tahap ini, Anda perlu mengimpor beberapa pustaka (library) Python yang dibutuhkan untuk analisis data dan pembangunan model machine learning.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

# **2. Memuat Dataset dari Hasil Clustering**

Memuat dataset hasil clustering dari file CSV ke dalam variabel DataFrame.

In [2]:
df = pd.read_csv(r'https://raw.githubusercontent.com/Jirrooo/beverage-sales-clustering-and-classification/refs/heads/main/Dataset_clustering.csv')

In [3]:
df.head()

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,Cluster
0,Female,21,1.62,64.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,0.0,1.0,no,Public_Transportation,1.0
1,Female,21,1.52,56.0,yes,no,3.0,3.0,Sometimes,yes,3.0,yes,3.0,0.0,Sometimes,Public_Transportation,1.0
2,Male,23,1.80,77.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,2.0,1.0,Frequently,Public_Transportation,1.0
3,Male,27,1.80,87.0,no,no,3.0,3.0,Sometimes,no,2.0,no,2.0,0.0,Frequently,Walking,0.0
4,Male,22,1.78,89.8,no,no,2.0,1.0,Sometimes,no,2.0,no,0.0,0.0,Sometimes,Public_Transportation,1.0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2087 entries, 0 to 2086
Data columns (total 17 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Gender                          2087 non-null   object 
 1   Age                             2087 non-null   int64  
 2   Height                          2087 non-null   float64
 3   Weight                          2087 non-null   float64
 4   family_history_with_overweight  2087 non-null   object 
 5   FAVC                            2087 non-null   object 
 6   FCVC                            2087 non-null   float64
 7   NCP                             2087 non-null   float64
 8   CAEC                            2087 non-null   object 
 9   SMOKE                           2087 non-null   object 
 10  CH2O                            2087 non-null   float64
 11  SCC                             2087 non-null   object 
 12  FAF                             20

# **3. Data Splitting**

Tahap Data Splitting bertujuan untuk memisahkan dataset menjadi dua bagian: data latih (training set) dan data uji (test set).

In [5]:
X = df.drop(columns=["Cluster"])
y = df["Cluster"]

In [13]:
data_combined = pd.concat([X, y], axis=1)

data_combined = data_combined.dropna(subset=[y.name])

X = data_combined.drop(columns=[y.name])
y = data_combined[y.name]

In [14]:
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_cols = X.select_dtypes(include=['object', 'bool']).columns.tolist()

In [15]:
# Preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(drop='first', sparse_output=False), categorical_cols)
    ]
)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# **4. Membangun Model Klasifikasi**


## **a. Membangun Model Klasifikasi**

Setelah memilih algoritma klasifikasi yang sesuai, langkah selanjutnya adalah melatih model menggunakan data latih.

Berikut adalah rekomendasi tahapannya.
1. Pilih algoritma klasifikasi yang sesuai, seperti Logistic Regression, Decision Tree, Random Forest, atau K-Nearest Neighbors (KNN).
2. Latih model menggunakan data latih.

In [16]:
# Random Forest
pipeline_rf = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])
pipeline_rf.fit(X_train, y_train)
y_pred_rf = pipeline_rf.predict(X_test)

# Logistic Regression
pipeline_lr = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(max_iter=1000, random_state=42))
])
pipeline_lr.fit(X_train, y_train)
y_pred_lr = pipeline_lr.predict(X_test)

- Random Forest Classifier adalah algoritma ensemble berbasis pohon keputusan yang bekerja dengan membuat banyak pohon keputusan saat pelatihan dan menggabungkan hasilnya (melalui voting) untuk meningkatkan akurasi dan mengurangi overfitting.

- Logistic Regression adalah algoritma klasifikasi linier yang digunakan untuk memprediksi probabilitas kelas. Meskipun sederhana, algoritma ini cukup efektif untuk dataset dengan hubungan linier antar fitur dan label target.

## **b. Evaluasi Model Klasifikasi**

Berikut adalah **rekomendasi** tahapannya.
1. Lakukan prediksi menggunakan data uji.
2. Hitung metrik evaluasi seperti Accuracy dan F1-Score (Opsional: Precision dan Recall).
3. Buat confusion matrix untuk melihat detail prediksi benar dan salah.

In [17]:
rf_acc = accuracy_score(y_test, y_pred_rf)
rf_f1 = f1_score(y_test, y_pred_rf, average='weighted')
lr_acc = accuracy_score(y_test, y_pred_lr)
lr_f1 = f1_score(y_test, y_pred_lr, average='weighted')

print("Random Forest")
print(f"Akurasi: {rf_acc:.4f}")
print(f"F1-Score: {rf_f1:.4f}\n")

print("Logistic Regression")
print(f"Akurasi: {lr_acc:.4f}")
print(f"F1-Score: {lr_f1:.4f}")

Random Forest
Akurasi: 0.7724
F1-Score: 0.7537

Logistic Regression
Akurasi: 0.7482
F1-Score: 0.6611


- Random Forest menghasilkan akurasi dan F1-score yang lebih tinggi dibandingkan Logistic Regression, menandakan performa yang lebih baik dalam memodelkan hubungan dalam data ini.

- Perbedaan F1-score yang signifikan menunjukkan bahwa Random Forest lebih baik dalam menangani distribusi kelas yang tidak seimbang atau kompleks, dibanding Logistic Regression yang mungkin kesulitan jika hubungan antar fitur tidak linier.

## **c. Tuning Model Klasifikasi (Optional)**

Gunakan GridSearchCV, RandomizedSearchCV, atau metode lainnya untuk mencari kombinasi hyperparameter terbaik

In [19]:
#Type your code here

## **d. Evaluasi Model Klasifikasi setelah Tuning (Optional)**

Berikut adalah rekomendasi tahapannya.
1. Gunakan model dengan hyperparameter terbaik.
2. Hitung ulang metrik evaluasi untuk melihat apakah ada peningkatan performa.

In [ ]:
#Type your code here

## **e. Analisis Hasil Evaluasi Model Klasifikasi**

Berikut adalah **rekomendasi** tahapannya.
1. Bandingkan hasil evaluasi sebelum dan setelah tuning (jika dilakukan).
2. Identifikasi kelemahan model, seperti:
  - Precision atau Recall rendah untuk kelas tertentu.
  - Apakah model mengalami overfitting atau underfitting?
3. Berikan rekomendasi tindakan lanjutan, seperti mengumpulkan data tambahan atau mencoba algoritma lain jika hasil belum memuaskan.

Kelemahan Model:

- Logistic Regression menunjukkan nilai F1-score yang cukup rendah (0.6611), kemungkinan karena tidak mampu menangkap kompleksitas pola dalam data.

- Analisis per kelas (jika dilakukan) bisa menunjukkan bahwa precision atau recall rendah pada kelas minoritas.

- Tidak ada indikasi langsung dari overfitting berdasarkan hasil, namun bisa jadi Random Forest cenderung overfit jika model terlalu kompleks dan tidak dibatasi parameternya seperti max_depth.

Rekomendasi Tindakan Lanjutan:

- Evaluasi per kelas menggunakan classification_report() untuk memahami precision dan recall tiap label.

- Coba algoritma lain seperti:
1. XGBoost atau Gradient Boosting untuk performa lebih tinggi dan kontrol overfitting.

2. K-Nearest Neighbors (KNN) jika data tidak terlalu besar, karena KNN bagus untuk pola lokal.

- Tuning hyperparameter menggunakan GridSearchCV atau RandomizedSearchCV agar model optimal.

- Lakukan feature selection atau analisis korelasi untuk mengurangi noise dari fitur yang tidak relevan.

- Kumpulkan data tambahan, terutama jika kelas tidak seimbang, untuk meningkatkan generalisasi model.